# Coupling MetaSWAP to MODFLOW 6 with BMI
This notebook can be used to reproduce published results for the coupling of MetaSWAP and MODFLOW 6, as reported in the MODFLOW 6 API paper (in progress).

MetaSWAP is a model for hydrologic processes in the unsaturated zone. Instead of doing a full solve of Richard's equation for each iteration in the coupling, it uses preprocessed steady state profiles for the moisture content. This way regional- and national-scale simulations are still relatively efficient and sufficiently detailed to account for the vertical processes such as capillary rise. More information on MetaSWAP (or SIMGRO, of which it is a component) can be found in "Quasi Steady‐State Simulation of the Unsaturated Zone in Groundwater Modeling of Lowland Regions" (https://doi.org/10.2136/vzj2007.0146) A coupling to MODFLOW 2005 already exists and is documented in "Integration of models using shared state variables: Implementation in the regional hydrologic modelling system SIMGRO" (https://doi.org/10.1016/j.jhydrol.2011.08.036). Note: the newly developed coupling to MODFLOW 6 does not conceptually differ from the latter.

## Supported operating systems
MetaSWAP is only available for the Windows operating system

## Prerequisites
To run the simulation and process the results, the following publicly available software and data are required:

* __libmf6.dll__ pre-compiled and available from https://github.com/MODFLOW-USGS/executables we have used version 6.2.2 for the results in the paper
* __MetaSWAP.dll__ (+3rd party dlls) pre-compiled available from the iMOD 5.2 release here: https://oss.deltares.nl/nl/web/imod. (NB: the MODFLOW 6 dll that comes with that release should be replaced with the version mentioned above)
* __imod_coupler__ is the coupling between MODFLOW 6 and MetaSWAP, sources are at https://github.com/Deltares/imod_coupler and it can be installed from pypi. Use version 0.10.0 to reproduce the published results
* __MetaSWAP database__ is the preprocessed table with steady-state moisture content profiles. It can be downloaded from the Wageningen ftp server: ftp://ftp.wur.nl/simgro/unsa/. We have used version 'LHM2018_v02vrz_BOFEK2012' for this simulation. Note that the database is large, many GBs, but it can be easily stripped from soil types not used in the simulation, reducing its size to roughly 200MB. For this case this would mean keeping the top-level directory '[1]' and discarding everything else.

Next to imod_coupler and its dependencies, you need to install the following Python packages from PyPi:
* __matplotlib__ a Python package required to generate the figures
* __pandas__ for storing the timeseries data below
* __flopy__ version 3.3.4 which works with the MODFLOW version mentioned above

## Model data and description
In summary, the coupled system is a 9x9 (3 layer) groundwater model, with boundary conditions (GHB) imposed on the cells in the first and last column and MetaSWAP cells coupled to all other MODFLOW cells:

<div>
<br/>
<img src="./model.png" width="400">
</div>

The figure marks the cell with SVAT id = 32 for which we plot some of the data below.

### MODFLOW 6
The MODFLOW 6 model is created with flopy (https://github.com/modflowpy/flopy) and is documented in another [notebook](build_mf6.ipynb) in this repository. 
    
### MetaSWAP

The MetaSWAP model data has been generated with iMOD 5.2 (https://oss.deltares.nl/nl/web/imod) and the input data is available in this repository in the folder [msw_data](./msw_data). Some features of the model follow here: 

##### Geometry
The extent of the model follows that of the groundwater model, except that the boundary colums have no matching MetaSWAP cell (SVAT). So the model has 63 horizontally independent SVATs covering the 9x7 MODFLOW cells in the top layer with all cell areas equal to 100 m$^2$. The soil surface sits at 0.0.

##### Soil data
The soil type is peat, type 1 of the BOFEK2012 specification for soil types ("BOFEK2012, de nieuwe, bodemfysische schematisatie van Nederland", Wageningen 2013, Alterra, Alterra-rapport 2387). Note the remark above: the database can be stripped from all other soil types to save disk space.

##### Vegetation
The land use type is agricultural with the crop species set to potatoes for all SVATs. Note that this has a relatively large seasonal transpiration rate. The irrigation period is set in the land use file [LUSE_SVAT.INP](./msw_data/LUSE_SVAT.INP) to run from day 150 to 240 for potatoes ('aardappelen'). 

##### Meteo
The meteo data, daily precipitation and reference evapotranspiration (Makkink) for the year 2018, an exceptionally dry year, comes from the Dutch national weather service (KNMI) for the 'De Bilt' station: [KNMI_20181231.txt](./preprocess/KNMI_20181231.txt). The python script [print_rain.py](./preprocess/print_rain.py) converts it to the MetaSWAP input file format.


### Coupling files: input for imod_coupler
We use imod_coupler to run the coupled system. The connection between MODFLOW cells and MetaSWAP SVATs is configured in the following mapping files:

* unsaturated zone flux from MetaSWAP to RCH: [rchindex2svat.dxc](./mf6_data/rchindex2svat.dxc)
* irrigation from WEL to MetaSWAP: [wellindex2svat.dxc](./mf6_data/wellindex2svat.dxc)
* storage coefficients (sc1) from MetaSWAP to STO: [nodenr2svat.dxc](./mf6_data/nodenr2svat.dxc)
* groundwater head from MODFLOW to MetaSWAP: [nodenr2svat.dxc](./mf6_data/nodenr2svat.dxc)

The MetaSWAP mapping from SVAT id (n,m) to an index in the exposed BMI arrays is generated from the file [mod2svat.inp](msw_data/mod2svat.inp)

---
**NOTE**

The *.dxc mapping files are static, so when the model grid is changed, locations of the head boundaries are altered, or, say, the iDOMAIN parameter is set for some cells in the top layer, the files likely need to be modified accordingly by hand.

---


## Running the simulation

We start by importing the necessary packages:

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from xmipy import XmiWrapper
from imod_coupler.metamod import MetaMod

## Setting up the coupled model

First we wrap both MetaSWAP and MODFLOW 6 with the XmiWrapper using the paths to their kernel dll, the model data directories, and optionally a path for library dependencies:

In [ ]:
# MetaSWAP
msw_dll = os.path.abspath("./imod_coupler/MetaSWAP.dll")
msw_dll_dir = os.path.abspath("./imod_coupler")
msw_dir = os.path.abspath("./msw_data")
msw_kernel = XmiWrapper(
    lib_path=msw_dll,
    lib_dependency=msw_dll_dir,
    working_directory=msw_dir
)

In [ ]:
# MODFLOW 6
mf6_dll = os.path.abspath("./imod_coupler/libmf6.dll")
mf6_dir = os.path.abspath("./mf6_data")
mf6_kernel = XmiWrapper(
    lib_path=mf6_dll,
    working_directory=mf6_dir
)

The coupling is then instantiated as

In [ ]:
# Set up the coupling
metamod = MetaMod(mf6=mf6_kernel, msw=msw_kernel, timing=False)

## Running the model
The MetaMod coupling object then has a BMI itself and running the model is therefore as straightforward as calling the initialize, update, finalize sequence just as you would for a single, isolated kernel. However, to collect some data for the plots below, we have inserted a few more lines of code.

In [ ]:
# Data for plotting
recharge = []
storage = []
head = []
sprinkling = []
times = []
head_3d = None

In [ ]:
# Initialize the coupling
metamod.initialize()

Now we can start the time loop and collect the data:

In [ ]:
# Run the time loop
start_time, current_time, end_time = metamod.get_times()
while current_time < end_time:
    current_time = metamod.update()
    
    # Add selected data
    head.append(metamod.msw_head[32])                  # internal MetaSWAP groundwater head for svat (32,1)
    recharge.append(metamod.mf6_recharge[31])          # the coupled recharge array from the RCH package, this is a volume
    storage.append(metamod.mf6_storage[40])            # the storage coefficients array (sc1)   
    sprinkling.append(metamod.mf6_sprinkling_wells[0]) # the sprinkling extraction from gw
    times.append(current_time)                         # time (day nr.)
    
    # Snapshot of head when sprinkling to show extraction cone
    if current_time == 151.0:
        head_3d = metamod.mf6_head.reshape(3,9,9).copy()
    
print('**The simulation has finished**')

In [ ]:
# Write timeseries data to file
df_coupling = pd.DataFrame({'time': times, 
                            'head': head, 
                            'recharge': recharge, 
                            'storage': storage, 
                            'sprinkling': sprinkling})
df_coupling.to_csv('./coupling_node41.csv', index=False)

# Clean up the resources
metamod.finalize()

and we're done running the simulation, all that's remaining is postprocess and interpret our results.

## Results

### Dynamics of the coupling

To illustrate the dynamics of the coupling, we are going to generate a timeseries plot of the coupled variables for the centermost cell, i.e. (1,5,5). Let's start by reading some fo the results back from file:

In [ ]:
# Load meteo input file
rain = np.loadtxt("./msw_data/METE_SVAT.INP")

# Load coupling variables
df_coupling = pd.read_csv("./coupling_node41.csv")

# Load MetaSWAP data
df_msw = pd.read_csv("./msw_data/svat_dtgw_0000000032.csv")
df_msw.columns = df_msw.columns.str.replace(' ', '')

Set uniform fig specs (USGS)

In [ ]:
import sys
sys.path.append('../common')
from figspecs import USGSFigure

fs = USGSFigure(figure_type="graph")

---
**NOTE**

The user specifies the recharge flux rate (L/T) in the RCH input file, inside MODFLOW 6 this is converted to a volumetric recharge rate (L$^{3}$/T). In the coupling, MetaSWAP provides 'vsim' which is a volume, and we divide that by the time step length (1.0 in this case) before it is assigned to the internal MODFLOW array. To compare this now more easily to the meteo data in the figure, we convert it back to a recharge flux rate in units (mm/d):

---

In [ ]:
rch_mm_per_day = 10.0*df_coupling['recharge']

Now we can generate the plot used in the API paper. We use the data for the cell with SVAT id = 32

In [ ]:
fig_width = 180 # mm
fig_width = fig_width / 10 / 2.54 # inches
fig_height = 0.6*fig_width
#fig_width = 8
#fig_height = 6
fig, axs = plt.subplots(3, sharex=True, sharey=False, figsize=(fig_width, fig_height))
box = dict(facecolor="none", edgecolor="none", pad=10, alpha=0.2)

# top panel
ln_rain = axs[0].plot(times, rain[:,2], label='rain', linewidth=0.75)
axs[0].set_ylabel('rain(mm/d)', bbox=box)
axs[0].set_xlim(1.0,365.0)
axs[0].set_ylim(0.0,30.0)


axs_ev = axs[0].twinx()
#axs_ev.plot(times, rain[:,3], 'g', label='ET ref.')
ln_tact = axs_ev.plot(times, -1.*df_msw['ETact(mm)'], 'red', label='$ET_{act}$', linewidth=0.75)
axs_ev.set_ylim(0.0,15.0)
fs.heading(axs[0], 'A')
axs_ev.set_ylabel('$ET_{\mathrm{act}}$ (mm/d)')

# added these three lines
lns = ln_rain + ln_tact
labs = [l.get_label() for l in lns]
axs[0].legend(lns, labs, loc=0)

# 2nd panel
axs[1].plot(times, rch_mm_per_day, label = 'recharge', linewidth=0.75)
axs[1].set_ylabel('$q_{\mathrm{rch}}$ (mm/d)', bbox=box)
axs[1].set_ylim(-2.0,20.0)
fs.heading(axs[1], 'B')

axs_ins = plt.axes([0.5,0.47,0.2,0.1])
axs_ins.plot(times, rch_mm_per_day, linewidth=0.75)
cap_rise = [min(x,0.0) for x in 10.0*df_coupling['recharge']]
axs_ins.fill_between(times, 0.0, cap_rise, color='lightgray', linewidth=0)
axs_ins.set_xlim(130.0,165.0)
axs_ins.set_ylim(-0.5,0.5)
plt.sca(axs_ins)
axs_ins.set_xticks([])


axs[1].indicate_inset_zoom(axs_ins)

# 3rd panel
axs[2].plot(times, df_coupling['head'], label = 'head', linewidth=0.75)
axs[2].set_ylabel('head (m)', bbox=box)
axs[2].set_ylim(-1.65,-0.35)
fs.heading(axs[2], 'C')

plt.sca(axs[2])
axs[2].set_xlim(1.0,365.0)
axs[2].set_xlabel('time (d)')
plt.xticks([1.0, 91.0, 182.0, 274.0, 366.0],
           ['Jan \'18','Apr \'18','Jul \'18','Oct \'18','Jan \'19'])
axs[2].set_xticks([32.0, 60.0, 121.0, 152.0, 213.0, 244.0, 305.0, 335.0], minor=True)

# align labels for y axis
fig.align_ylabels(axs)

fig.show()
fig.savefig("coupling-mf6-msw.png")

### Extraction cone
The following plot shows the extraction cone in the deepest layer which occurs when the MetaSWAP simulation starts pumping up groundwater for agricultural demand:

In [ ]:
plt.imshow(head_3d[2, :, :])
plt.show()

### Transpiration and irrigation

The following analysis shows the correlation between transpiration and irrigation

In [ ]:
# Load MetaSWAP data for unit coupled to (1,5,5)
df = pd.read_csv("./msw_data/svat_dtgw_0000000032.csv")
df.columns = df.columns.str.replace(' ', '')

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(times, -1.*df_msw['Tact(mm)'], label="transpiration")
sprinkling_arr = np.array(sprinkling)
plt.plot(times, -0.04*sprinkling_arr, label="irrigation extr (x0.04)")
plt.xticks([1.0, 91.0, 182.0, 274.0, 365.0],
           ['Jan','Apr','Jul','Oct','Dec'])
plt.xlim(91.0,274.0)
plt.legend()